# U.S. Census Income Differential Privacy

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import wasserstein_distance
from opendp.mod import enable_features
from opendp.measurements import make_laplace
from opendp.domains import atom_domain
from opendp.metrics import absolute_distance
enable_features("contrib")

In [2]:
df_census = pd.read_csv('../data/adult.csv')
df_census.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


### Clean Data - Remove "?"

In [3]:
question_mark_counts = (df_census == "?").sum()
print(question_mark_counts)

age                  0
workclass         1836
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     583
income               0
dtype: int64


In [ ]:
df_cleaned = df_census[~df_census.applymap(lambda x: str(x).strip() == "?").any(axis=1)]
df_cleaned = df_cleaned.sample(n=5000, random_state=72)

C:\Users\elzha\AppData\Local\Temp\ipykernel_20864\2381018209.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df_census[~df_census.applymap(lambda x: str(x).strip() == "?").any(axis=1)]


### Setup Epsilon Values and Add DP

In [11]:
# Selecting values for epsilon
# Low ε (e.g., 0.1, 0.5) → More noise, stronger privacy, but lower accuracy.
# High ε (e.g., 5, 10) → Less noise, higher accuracy, but weaker privacy.

age = df_cleaned["age"].astype(float).tolist()
hours_per_week = df_cleaned["hours.per.week"].astype(float).tolist()
education_num = df_cleaned["education.num"].astype(float).tolist()
capital_gain = df_cleaned["capital.gain"].astype(float).tolist()
capital_loss = df_cleaned["capital.loss"].astype(float).tolist()

delta_1 = np.max(np.abs(np.diff(np.sort(age)))) 
delta_2 = np.max(np.abs(np.diff(np.sort(hours_per_week))))
delta_3 = np.max(np.abs(np.diff(np.sort(education_num))))
delta_4 = np.max(np.abs(np.diff(np.sort(capital_gain))))
delta_5 = np.max(np.abs(np.diff(np.sort(capital_loss))))

epsilon_values = [0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
results_age = {}
results_hours_per_week = {}
results_education_num = {}
results_capital_gain = {}
results_capital_loss = {}

In [12]:
# adding dp to age
for epsilon in epsilon_values:
    scale = delta_1 / epsilon
    laplace_mechanism = make_laplace(
        input_domain=atom_domain(T=float),
        input_metric=absolute_distance(T=float),
        scale=scale
    )
    dp_samples = np.array([[laplace_mechanism(x) for x in age] for _ in range(10)])
    dp_age = np.mean(dp_samples, axis=0)
    mae = np.mean(np.abs(dp_age - age))
    stability = np.mean(np.std(dp_samples, axis=0))  
    noise_variance = np.var(dp_samples - age, axis=0).mean()
    emd = wasserstein_distance(age, dp_age)
    results_age[epsilon] = {"dp_values": dp_age, "mae": mae, "emd": emd, "stability": stability,
        "noise_variance": noise_variance}

# adding dp for hours per week
for epsilon in epsilon_values:
    scale = delta_2 / epsilon
    laplace_mechanism = make_laplace(
        input_domain=atom_domain(T=float),
        input_metric=absolute_distance(T=float),
        scale=scale
    )
    dp_samples = np.array([[laplace_mechanism(x) for x in hours_per_week] for _ in range(10)])
    dp_hours_per_week = np.mean(dp_samples, axis=0)
    mae = np.mean(np.abs(dp_hours_per_week - hours_per_week))
    stability = np.mean(np.std(dp_samples, axis=0))  
    noise_variance = np.var(dp_samples - hours_per_week, axis=0).mean()
    emd = wasserstein_distance(hours_per_week, dp_hours_per_week)
    results_hours_per_week[epsilon] = {"dp_values": dp_hours_per_week, "mae": mae, "emd": emd, "stability": stability,
        "noise_variance": noise_variance}

# adding dp for education num
for epsilon in epsilon_values:
    scale = delta_3 / epsilon
    laplace_mechanism = make_laplace(
        input_domain=atom_domain(T=float),
        input_metric=absolute_distance(T=float),
        scale=scale
    )
    dp_samples = np.array([[laplace_mechanism(x) for x in education_num] for _ in range(10)])
    dp_education_num = np.mean(dp_samples, axis=0)
    mae = np.mean(np.abs(dp_education_num - education_num))
    stability = np.mean(np.std(dp_samples, axis=0))  
    noise_variance = np.var(dp_samples - education_num, axis=0).mean()
    emd = wasserstein_distance(education_num, dp_education_num)
    results_education_num[epsilon] = {"dp_values": dp_education_num, "mae": mae, "emd": emd, "stability": stability,
        "noise_variance": noise_variance}

# adding dp for capital gain
for epsilon in epsilon_values:
    scale = delta_4 / epsilon
    laplace_mechanism = make_laplace(
        input_domain=atom_domain(T=float),
        input_metric=absolute_distance(T=float),
        scale=scale
    )
    dp_samples = np.array([[laplace_mechanism(x) for x in capital_gain] for _ in range(10)])
    dp_capital_gain = np.mean(dp_samples, axis=0)
    mae = np.mean(np.abs(dp_capital_gain - capital_gain))
    stability = np.mean(np.std(dp_samples, axis=0))  
    noise_variance = np.var(dp_samples - capital_gain, axis=0).mean()
    emd = wasserstein_distance(capital_gain, dp_capital_gain)
    results_capital_gain[epsilon] = {"dp_values": dp_capital_gain, "mae": mae, "emd": emd, "stability": stability,
        "noise_variance": noise_variance}

# adding dp for capital loss
for epsilon in epsilon_values:
    scale = delta_5 / epsilon
    laplace_mechanism = make_laplace(
        input_domain=atom_domain(T=float),
        input_metric=absolute_distance(T=float),
        scale=scale
    )
    dp_samples = np.array([[laplace_mechanism(x) for x in capital_loss] for _ in range(10)])
    dp_capital_loss = np.mean(dp_samples, axis=0)
    mae = np.mean(np.abs(dp_capital_loss - capital_loss))
    stability = np.mean(np.std(dp_samples, axis=0))  
    noise_variance = np.var(dp_samples - capital_loss, axis=0).mean()
    emd = wasserstein_distance(capital_loss, dp_capital_loss)
    results_capital_loss[epsilon] = {"dp_values": dp_capital_loss, "mae": mae, "emd": emd, "stability": stability,
        "noise_variance": noise_variance}

In [ ]:
# saving results so I don't have to rerun
import json

# Save the results for each column as a JSON file
for epsilon in results_age:
    results_age[epsilon]["dp_values"] = results_age[epsilon]["dp_values"].tolist()
    
with open("../data/results_age.json", "w") as f:
    json.dump(results_age, f)

for epsilon in results_hours_per_week:
    results_hours_per_week[epsilon]["dp_values"] = results_hours_per_week[epsilon]["dp_values"].tolist()

with open("../data/results_hours.json", "w") as f:
    json.dump(results_hours_per_week, f)

for epsilon in results_education_num:
    results_education_num[epsilon]["dp_values"] = results_education_num[epsilon]["dp_values"].tolist()

with open("../data/results_education.json", "w") as f:
    json.dump(results_education_num, f)

for epsilon in results_capital_gain:
    results_capital_gain[epsilon]["dp_values"] = results_capital_gain[epsilon]["dp_values"].tolist()

with open("../data/results_cap_gain.json", "w") as f:
    json.dump(results_capital_gain, f)

for epsilon in results_capital_loss:
    results_capital_loss[epsilon]["dp_values"] = results_capital_loss[epsilon]["dp_values"].tolist()

with open("../data/results_cap_loss.json", "w") as f:
    json.dump(results_capital_loss, f)

### Visualizations

In [ ]:
# Histograms
